<a href="https://colab.research.google.com/github/NadirYasar/Dog_Food_Analysis/blob/main/DOG_FOOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
!pip install findspark

import findspark
findspark.init()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
from google.colab import files

# Dosya yükleme
uploaded = files.upload()


Saving dog_food.csv to dog_food (1).csv


In [ ]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("DogFoodAnalysis").getOrCreate()



In [ ]:
data = spark.read.csv('/content/dog_food.csv', inferSchema=True, header=True)
data.show()


+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [ ]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [ ]:
from pyspark.ml.feature import VectorAssembler


In [ ]:
features_col = ['A','B','C','D']

In [ ]:
assembler = VectorAssembler(inputCols= ['A','B','C','D'], outputCol='features')

In [ ]:
output = assembler.transform(data)

In [ ]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
final_data = output.select('features','Spoiled')


In [ ]:
train_data, test_data = output.randomSplit([0.8, 0.2], seed=42)

In [ ]:
rfc = RandomForestClassifier(labelCol = 'Spoiled', featuresCol ='features', numTrees= 100)

In [ ]:
rf_model = rfc.fit(train_data)

In [ ]:
predictions = rf_model.transform(test_data)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='Spoiled', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f"Model doğruluğu: {accuracy}")

Model doğruluğu: 0.9863013698630136


In [ ]:
importances = rf_model.featureImportances
for i, importance in enumerate(importances):
    print(f"Özellik {features_col[i]}: {importance}")

Özellik A: 0.017076186758210314
Özellik B: 0.024112325095806162
Özellik C: 0.9353926799165766
Özellik D: 0.023418808229406923
